In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# gets reviews for a single game
def get_reviews(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()

In [2]:
# gets n reviews for a single game
def get_n_reviews(appid, n=100):
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }

    while n > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, n)
        n -= 100

        response = get_reviews(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']

        if len(response['reviews']) < 100: break

    return reviews

In [17]:
# gets n appids (games)
def get_n_appids(n=100, filter_by='topsellers'):
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    page = 0

    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])

    return appids[:n]

In [8]:
reviews = []
appids = get_n_appids(750)
for appid in appids:
    reviews += get_n_reviews(appid, 100)
df = pd.DataFrame(reviews)[['review', 'voted_up']]

,review,voted_up
0,most of these negative reviews are based on bu...,False
1,"TLDR: If not for their monetization scheme, th...",False
2,I understand that some people are upset give t...,True
3,Pros:\n\n- Probably the best optimization in a...,True
4,I came into this game expecting basically noth...,True
...,...,...
71797,"You should buy this game, Now!\n\nIf you have ...",True
71798,"If you like Overcooked, you'll love this one. ...",True
71799,I'm not going to lie.\nThis game is so wholeso...,True
71800,Super cute game! Online multiplayer is easy to...,True


In [16]:
df["voted_up"] = df["voted_up"].astype(int) # converts True/False to 1/0
df2 = df.iloc[65000:] # slice dataset for new reviews to test later 

In [ ]:
df.to_csv('~/Documents/steam-reviewer-bert/data/reviews.csv')
df2.to_csv('~/Documents/steam-reviewer-bert/data/new_reviews.csv')